# Install

In [1]:
!pip install -U lightautoml

# Library

In [2]:
import pandas as pd
from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML

# Load

In [3]:
train_data = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
samp_sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

# Parameter

In [4]:
lgb_params = {
    'metric': 'RMSE',
    'lambda_l1': 1e-07,
    'lambda_l2': 2e-07,
    'num_leaves': 42,
    'feature_fraction': 0.55,
    'bagging_fraction': 0.9,
    'bagging_freq': 3,
    'min_child_samples': 19,
    'num_threads': 4
}

cb_params = {
    'num_trees': 10000,
    'od_wait': 1200,
    'learning_rate': 0.001,
    'l2_leaf_reg': 64,
    'subsample': 0.83,
    'random_strength': 17.17,
    'max_depth': 10,
    'min_data_in_leaf': 10,
    'leaf_estimation_iterations': 3,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'bootstrap_type': 'Bernoulli',
    'leaf_estimation_method': 'Newton',
    'random_seed': 42,
    "thread_count": 4
}

# Train

In [5]:
automl = TabularAutoML(task = Task('reg', ),
                       timeout = 3600,
                       cpu_limit = 4,
                       reader_params = {'n_jobs': 4, 'cv': 10, 'random_state': 42},
                       general_params = {'use_algos': [['lgb', 'cb']]},
                       lgb_params = {'default_params': lgb_params, 'freeze_defaults': True},
                       cb_params = {'default_params': cb_params, 'freeze_defaults': True},
                       verbose = 2
                      )

pred = automl.fit_predict(train_data, roles = {'target': 'loss', 'drop': ['id']})

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


Start automl preset with listed constraints:
- time: 3600 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (250000, 102)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 3556.57466173172 secs
Start fitting Selector_LightGBM ...

===== Start working with fold 0 for Selector_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's l2: 62.4472
[200]	valid's l2: 62.2395
[300]	valid's l2: 62.1806
[400]	valid's l2: 62.1664
[500]	valid's l2: 62.15
[600]	valid's l2: 62.1911
Early stopping, best iteration is:
[515]	valid's l2: 62.1499
Selector_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's l2: 62.4719
[200]	valid's l2: 62.2275
[300]	valid's l2: 62.1717
[400]	valid's l2: 62.1489
[500]	valid's l2: 62.126
Early stopping,

# Predict

In [6]:
test_pred = automl.predict(test_data)

# Save

In [7]:
samp_sub['loss'] = test_pred.data[:, 0]
samp_sub.to_csv('submission.csv', index = False)